In [1]:
#!pip install selenium

     |████████████████████████████████| 993kB 2.7MB/s eta 0:00:01
     |████████████████████████████████| 368kB 4.1MB/s eta 0:00:01
     |████████████████████████████████| 4.1MB 3.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 4.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 2.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/71/52/39d20e03abd0ac9159c162ec24b93fbcaa111e8400308f2465432495ca2b/async_generator-1.10-py3-none-any.whl
     |████████████████████████████████| 512kB 2.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 209kB/s eta 0:00:01
     |████████████████████████████████| 122kB 858kB/s eta 0:00:01
  Running setup.py install for cffi ... done
You should consider upgrading via the 'pip install --upgrade pip' command.


In [49]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from time import sleep


class LobbyistNsw:
    def __init__(self, name, abn, trading_name, on_watch_list, status, details_anchor):
        self.name = name
        self.abn = abn
        self.trading_name = trading_name
        self.on_watch_list = on_watch_list
        self.status = status
        self.details_anchor = details_anchor
    
    @property
    def clients(self):
        return self._clients

    @clients.setter
    def clients(self, value):
        self._clients = value

class LobbyistNsw_Client:
    def __init__(self, name, abn, active, foreign_principal, countries, date_added):
        self.name = name
        self.abn = abn
        self.active = active
        self.foreign_principal = foreign_principal
        self.countries = countries
        self.date_added = date_added



class LobbyistNswScrapper:
    def __init__(self):
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        self.driver = webdriver.Chrome(options = chrome_options)
        self.driver.get("https://lobbyists.elections.nsw.gov.au/whoisontheregister")

    def get_lobbyists(self):
        lobbyists = []
        dataTableElement = self.driver.find_element(By.CLASS_NAME, 'dataTable')
        rows = dataTableElement.find_elements(By.CSS_SELECTOR, 'tbody tr[role="row"]')
        for row in rows:
            columns = row.find_elements(By.CSS_SELECTOR, "*")
            anchor = columns[0].find_element(By.TAG_NAME, 'a')
            lobbyist = LobbyistNsw(
                name = columns[0].text,
                abn = columns[1].text,
                trading_name = columns[2].text,
                on_watch_list = columns[3].text,
                status = columns[4].text,
                details_anchor = anchor
            )
            lobbyists.append(lobbyist)
        return lobbyists

    def populate_details(self, lobbyist: LobbyistNsw):
        lobbyist.clients = []
        lobbyist.details_anchor.click()
        sleep(1)
        modalBody = self.driver.find_element(By.CLASS_NAME, 'modal-body')
        rows = modalBody.find_elements(By.CSS_SELECTOR, '#CLI .dataTable tbody tr')
        for row in rows:
            columns = row.find_elements(By.CSS_SELECTOR, 'td')
            activeImg = row.find_element(By.CLASS_NAME, 'checkImg')
            abnElement = columns[1].find_element(By.CSS_SELECTOR, 'span[id^="j_id"] a')
            foreignPrincipalElement = columns[3]
            countriesElement = columns[4]
            client = LobbyistNsw_Client(
                name = columns[0].get_attribute('innerHTML'),
                abn = abnElement.get_attribute('innerHTML'),
                active = activeImg.get_attribute('alt') == 'Checked',
                foreign_principal = foreignPrincipalElement.text,
                countries = countriesElement.text,
                date_added = columns[5].get_attribute('innerHTML')
            )
            lobbyist.clients.append(client)

    def close(self):
        self.driver.quit()


In [50]:
import json
scrapper = LobbyistNswScrapper()
lobbyists = scrapper.get_lobbyists()

scrapper.populate_details(lobbyists[0])

for client in lobbyists[0].clients:
    print(f"{json.dumps(client.__dict__)}")


scrapper.close()

AUSTRALIA CHINA BUSINESS COUNCIL
AUSTRALIAN FUNERAL DIRECTORS ASSOCIATION LTD
INVOCARE LIMITED
PARED LTD
{"name": "AUSTRALIA CHINA BUSINESS COUNCIL", "abn": "57075909625", "active": true, "foreign_principal": "", "countries": "", "date_added": "06 Jan 2022"}
{"name": "AUSTRALIAN FUNERAL DIRECTORS ASSOCIATION LTD", "abn": "33007331580", "active": true, "foreign_principal": "", "countries": "", "date_added": "06 Jan 2022"}
{"name": "INVOCARE LIMITED", "abn": "42096437393", "active": true, "foreign_principal": "", "countries": "", "date_added": "06 Jan 2022"}
{"name": "PARED LTD", "abn": "40002343615", "active": true, "foreign_principal": "", "countries": "", "date_added": "06 Jan 2022"}


In [48]:
import json
for client in lobbyists[0].clients:
    print(f"{json.dumps(client.__dict__)}")

{"name": "AUSTRALIA CHINA BUSINESS COUNCIL", "abn": "57075909625", "active": true, "foreign_principal": true, "countries": "", "date_added": "06 Jan 2022"}
{"name": "AUSTRALIAN FUNERAL DIRECTORS ASSOCIATION LTD", "abn": "33007331580", "active": true, "foreign_principal": true, "countries": "", "date_added": "06 Jan 2022"}
{"name": "INVOCARE LIMITED", "abn": "42096437393", "active": true, "foreign_principal": true, "countries": "", "date_added": "06 Jan 2022"}
{"name": "PARED LTD", "abn": "40002343615", "active": true, "foreign_principal": true, "countries": "", "date_added": "06 Jan 2022"}


## Using Google to Search LinkedIn

In [15]:
#!pip install requests

In [20]:
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
# This bit of code allows me to import modules for adjacent folders:
import os
import sys
import requests as re
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.secrets import secrets as sc
import json

In [56]:
def custom_search(search_term, start = 1):
    custom_search_url = f"https://www.googleapis.com/customsearch/v1?key={sc.google_customsearch_api_key}&cx={sc.google_programmable_search_engine_id}&q={search_term}&start={start}"
    result = re.get(custom_search_url)
    return result.json()

def print_results(results):
    for item in results['items']:
        print(item['title'])
        print(item['link'])

This is what my profile link looks like:
https://www.linkedin.com/in/michael-gordon-b6698051/

In [73]:
mgordon_result = custom_search('"Michael Gordon" senior software LinkedIn', 21)
print_results(mgordon_result)

Sviatoslav Didukh - Associate Director, Experience Design - EPAM ...
https://ua.linkedin.com/in/sviatoslavdidukh
Michael Gordon | Harrison Street
https://www.harrisonst.com/leadership/michael-gordon/
Michael Gordon - Business Consultant - Self Employed | LinkedIn
https://tt.linkedin.com/in/michael-gordon-3b4a4723
Our Team - Group Gordon
https://groupgordon.com/people/
100+ perfiles de «Michael Gordon» | LinkedIn
https://bo.linkedin.com/pub/dir/Michael/Gordon/ca-0-Canada
Team | Angelo Gordon
https://www.angelogordon.com/team/
Michael Gordon - New Ireland Assurance - LinkedIn
https://ie.linkedin.com/in/michaelgordonmjg
NJDEP New Jersey Department of Environmental Protection
https://www.nj.gov/dep/easyaccess/commissoffice.htm
Amrita Samantasinghar - Software Engineer, Sr I - LinkedIn
https://in.linkedin.com/in/amrita-samantasinghar
Michael Gordon - Tshwane University of Technology - LinkedIn
https://za.linkedin.com/in/michael-gordon-b0831521


In [67]:
namin_result = custom_search('"Naeer Amin" LinkedIn')
print_results(namin_result)

Alyssa Clark - Administrative Coordinator - UPASS, HELPS - LinkedIn
https://au.linkedin.com/in/alyssa-clark-428588192
Jung Gyu Park - Software Engineer - SafetyCulture | LinkedIn
https://au.linkedin.com/in/jung-gyu-park-5b01a21b
Ciaran Cullen - Head of Talent - SafetyCulture | LinkedIn
https://au.linkedin.com/in/ciarancullen
Shuvra Sikder - Camden Council NSW Australia - LinkedIn
https://au.linkedin.com/in/shuvra-sikder-722126196
Anthony Muscat - Senior Frontend Developer - Pearson | LinkedIn
https://au.linkedin.com/in/anthony-muscat-b829676


In [68]:
schemweno_result = custom_search('"Sharon Chemweno" LinkedIn')
print_results(schemweno_result)

Sharon Chemweno - Ben-Gurion University of the Negev - LinkedIn
https://ke.linkedin.com/in/sharon-chemweno-b21b8517a
Ruth Cherogony - Project Intern - LinkedIn
https://il.linkedin.com/in/ruth-cherogony-749692234
100+ perfiles de «Chemweno» | LinkedIn
https://bo.linkedin.com/pub/dir/+/Chemweno/ke-0-Kenya
Jacob Maundu - Resolution Insurance Kenya - LinkedIn
https://ke.linkedin.com/in/jacob-maundu-5433a0173
Tania Acuna - Ben Gurion University of the Negev - LinkedIn
https://il.linkedin.com/in/tania-acuna-9a51a7b3
Beatrice Wambui - Case Manager - RefugePoint | LinkedIn
https://www.linkedin.com/in/beatrice-wambui-844855ba
Felecien NDAYISHIMIYE - WUA Technician - LinkedIn Rwanda
https://rw.linkedin.com/in/felecien-ndayishimiye-4a687a206
Jason Dorey - Director of Academics - CanGlory Education | LinkedIn
https://ca.linkedin.com/in/jason-dorey-b10308108
Pauline Ang'ang'o - Managing Editor - Genesis Tech | LinkedIn
https://hu.linkedin.com/in/pauline-ang-ang-o-194b55128
Mayumi Evangelista - Imme

In [69]:
gbohorquez_result = custom_search('"Gerardo Bohorquez Restrepo" LinkedIn')
print_results(gbohorquez_result)

Gerardo Restrepo - Senior Account Executive - LinkedIn
https://www.linkedin.com/in/gerardo-restrepo-7239757
Gerardo Restrepo - Professeur - Université de Sherbrooke | LinkedIn
https://ca.linkedin.com/in/gerardo-restrepo-675aa236
Gerardo Bohorquez - Colombia | Perfil profesional | LinkedIn
https://co.linkedin.com/in/gerardo-bohorquez-a7676290
40+ perfiles de «Bohorquez» | LinkedIn
https://bo.linkedin.com/pub/dir/+/Bohorquez/au-0-Australia
F5 group F5 group - Marketing Manager - F5 group | LinkedIn
https://co.linkedin.com/in/f5-group-f5-group-648a591b
Passiona Cottee - A/Director - LinkedIn
https://au.linkedin.com/in/passiona
Understanding Hierarchies using Dendrograms | by Yash Gupta ...
https://medium.com/dssimplified/understanding-hierarchies-using-dendrograms-e3aef7ac5ea4
